In [ ]:
import ads
import oci
import requests
import tempfile

from ads.model.framework.sklearn_model import SklearnModel
from oci.signer import Signer
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

ads.set_auth(auth="resource_principal")

# Load dataset and Prepare train and test split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Train a LogisticRegression model
sklearn_estimator = LogisticRegression()
sklearn_estimator.fit(X_train, y_train)

In [ ]:
# Instantiate ads.model.framework.sklearn_model.SklearnModel using the sklearn LogisticRegression model
sklearn_model = SklearnModel(
    estimator=sklearn_estimator, artifact_dir=tempfile.mkdtemp()
)

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

In [ ]:
# Autogenerate score.py, serialized model, runtime.yaml, input_schema.json and output_schema.json
#sklearn_model.prepare(
#    inference_conda_env="dbexp_p38_cpu_v1",
#    X_sample=X_train,
#    y_sample=y_train,
#)

sklearn_model.prepare(
    inference_conda_env="automlx251_p311_cpu_x86_64_v2",
    X_sample=X_train,
    y_sample=y_train,
    force_overwrite=True,
)

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

In [ ]:
# Verify generated artifacts
sklearn_model.verify(X_test)

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

In [ ]:
# Register scikit-learn model
model_id = sklearn_model.save(display_name="DIY SKLearn Model For Iris Classification")

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

In [ ]:
# Deploy and create an endpoint for the SKLearn model
sklearn_model.deploy(
    display_name="DIY SKLearn Model For Iris Classification",
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=1,
    deployment_memory_in_gbs=16,
)
print(f"Endpoint: {sklearn_model.model_deployment.url}")

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

In [ ]:
# The OCI SDK must be installed for this example to function properly.
# Installation instructions can be found here: https://docs.cloud.oracle.com/en-us/iaas/Content/API/SDKDocs/pythonsdk.htm
config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
  tenancy=config['tenancy'],
  user=config['user'],
  fingerprint=config['fingerprint'],
  private_key_file_location=config['key_file'],
  pass_phrase=config['pass_phrase'])

endpoint = sklearn_model.model_deployment.url + '/predict'
body = {"data": [[5.1, 3.5, 1.4, 0.2], [5.0, 2.0, 3.5, 1.0], [6.3, 3.3, 6.0, 2.5]]} # payload goes here
headers = {} # header goes here

print("Response from deployed model follows...")
requests.post(endpoint, json=body, auth=auth, headers=headers).json()

In [ ]:
# Alternately, the OCI CLI may be invoked with the command line
print("Use the following command line to invoke the deployment via the OCI CLI...\n")
print("oci raw-request --http-method POST --target-uri " + sklearn_model.model_deployment.url + "/predict --request-body '{\"data\": [[5.1, 3.5, 1.4, 0.2], [5.0, 2.0, 3.5, 1.0], [6.3, 3.3, 6.0, 2.5]]}'")